In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import warnings
import datetime

# Suppress warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

2025-04-15 16:45:01.370962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 16:45:01.465457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 16:45:01.465993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 16:45:01.579697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 16:45:02.934527: W tensorflow/compiler/tf

In [2]:
# 1. Data Loading and Preprocessing
def load_data(file_path):
    """
    Load and preprocess the electrical load data.
    """
    try:
        # Load the data
        data = pd.read_csv(file_path)
        
        print(f"Data loaded successfully with {data.shape[0]} rows and {data.shape[1]} columns.")
        
        # Extract hour from datetime
        if 'Datetime' in data.columns:
            data['Datetime'] = pd.to_datetime(data['Datetime'])
            data['Hour'] = data['Datetime'].dt.hour
            data['Date'] = data['Datetime'].dt.date
        
        # Print some basic statistics
        print("\nData summary:")
        print(f"Time period: {data['Datetime'].min()} to {data['Datetime'].max()}")
        print(f"States in the dataset: {data['State'].unique()}")
        print(f"Load value range: {data['Value'].min()} to {data['Value'].max()}")
        
        return data
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None


In [14]:
# 2. LSTM Model for Load Prediction
def create_dataset(dataset, time_step=1):
    """
    Create time series dataset for LSTM.
    """
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

def build_lstm_model(data, state_name, time_step=24):
    """
    Build and train an LSTM model for load forecasting.
    """
    try:
        # Filter data for the specific state
        state_data = data[data['State'] == state_name]['Value'].values
        if len(state_data) == 0:
            print(f"No data found for state: {state_name}")
            print(f"Available states: {data['State'].unique()}")
            return None, None
            
        state_data = state_data.reshape(-1, 1)
        
        # Normalize the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        state_data = scaler.fit_transform(state_data)
        
        # Split into train and test sets
        train_size = int(len(state_data) * 0.8)
        train_data = state_data[0:train_size, :]
        test_data = state_data[train_size:len(state_data), :]
        
        # Reshape into X=t and Y=t+1
        X_train, y_train = create_dataset(train_data, time_step)
        X_test, y_test = create_dataset(test_data, time_step)
        
        # Reshape input to be [samples, time steps, features]
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        
        # Create the LSTM model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))
        
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        # Train the model with early stopping
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
        model.fit(
            X_train, y_train,
            batch_size=32,
            epochs=100,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[callback]
        )
        
        # Make predictions
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)
        
        # Invert predictions to original scale
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        y_train_inv = scaler.inverse_transform([y_train])
        y_test_inv = scaler.inverse_transform([y_test])
        
        # Calculate metrics
        train_rmse = np.sqrt(mean_squared_error(y_train_inv[0], train_predict[:,0]))
        test_rmse = np.sqrt(mean_squared_error(y_test_inv[0], test_predict[:,0]))
        train_r2 = r2_score(y_train_inv[0], train_predict[:,0])
        test_r2 = r2_score(y_test_inv[0], test_predict[:,0])
        
        print(f'State: {state_name}')
        print(f'Train RMSE: {train_rmse:.2f}, Test RMSE: {test_rmse:.2f}')
        print(f'Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')
        
        return model, scaler
    
    except Exception as e:
        print(f"Error building LSTM model: {e}")
        return None, None

def predict_next_day_load(model, scaler, recent_data, time_step=24):
    """
    Predict load for the next 24 hours using the LSTM model.
    """
    try:
        # Prepare the input sequence for prediction
        recent_data = recent_data.reshape(-1, 1)
        recent_data = scaler.transform(recent_data)
        
        # Reshape for LSTM input
        X_input = recent_data.reshape(1, time_step, 1)
        
        # Get LSTM prediction
        prediction = model.predict(X_input)
        prediction = scaler.inverse_transform(prediction)[0, 0]
        
        return prediction
    
    except Exception as e:
        print(f"Error in prediction: {e}")
        return None


In [7]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

def create_load_fuzzy_system(min_load, max_load):
    """
    Create a fuzzy inference system for classifying load values.
    Returns the fuzzy variable and a function to get the label.
    """
    try:
        # Define input
        load = ctrl.Antecedent(np.arange(min_load, max_load + 1, 1), 'load')

        # Membership functions
        load['very_low'] = fuzz.trimf(load.universe, [min_load, min_load, min_load + (max_load - min_load) * 0.25])
        load['low'] = fuzz.trimf(load.universe, [min_load, min_load + (max_load - min_load) * 0.25, min_load + (max_load - min_load) * 0.5])
        load['medium'] = fuzz.trimf(load.universe, [min_load + (max_load - min_load) * 0.25, min_load + (max_load - min_load) * 0.5, min_load + (max_load - min_load) * 0.75])
        load['high'] = fuzz.trimf(load.universe, [min_load + (max_load - min_load) * 0.5, min_load + (max_load - min_load) * 0.75, max_load])
        load['very_high'] = fuzz.trimf(load.universe, [min_load + (max_load - min_load) * 0.75, max_load, max_load])

        # Return the Antecedent object and a function to infer label
        def infer_load_label(value):
            memberships = {label: fuzz.interp_membership(load.universe, load[label].mf, value)
                           for label in load.terms}
            best_label = max(memberships, key=memberships.get)
            return best_label
        
        return load, infer_load_label

    except Exception as e:
        print(f"Error creating fuzzy system: {e}")
        return None, lambda x: "unknown"


In [11]:
# 4. Main Forecasting Function
def forecast_load_for_state(data, state_name, time_step=24):
    """
    Forecast load for a specific state using the sequential LSTM + Fuzzy Logic approach.
    """
    try:
        # Get state data
        state_data = data[data['State'] == state_name]['Value'].values
        
        # Get min and max load for fuzzy system
        min_load = np.min(state_data) * 0.9  # Add some margin
        max_load = np.max(state_data) * 1.1
        
        # Build LSTM model
        print(f"\nBuilding LSTM model for {state_name}...")
        model, scaler = build_lstm_model(data, state_name, time_step)
        
        if model is None or scaler is None:
            print(f"Failed to build model for {state_name}. Skipping.")
            return None
        
        # Create fuzzy system
        print(f"\nCreating fuzzy logic system for {state_name}...")
        fuzzy_system = create_load_fuzzy_system(min_load, max_load)
        
        if fuzzy_system is None:
            print(f"Failed to create fuzzy system for {state_name}. Using only LSTM predictions.")
        
        # Get recent data for prediction
        recent_data = state_data[-time_step:]
        
        # Forecast for next 24 hours
        lstm_predictions = []
        fuzzy_predictions = []
        
        print(f"\nForecasting load for next 24 hours for {state_name}...")
        for hour in range(24):
            # Get LSTM prediction
            lstm_pred = predict_next_day_load(model, scaler, recent_data, time_step)
            lstm_predictions.append(lstm_pred)
            
            # Update recent data for next prediction (rolling forecast)
            recent_data = np.append(recent_data[1:], [lstm_pred])

        load_system, get_label = create_load_fuzzy_system(min_load, max_load)
        
        fuzzystrs = [get_label(val) for val in lstm_predictions]
        # # Plot results
        # plt.figure(figsize=(12, 6))
        # plt.plot(range(24), lstm_predictions, 'b-', label='LSTM Predictions')
        # plt.plot(range(24), fuzzy_predictions, 'r-', label='Fuzzy Logic Refined')
        # plt.title(f'Load Forecast for {state_name} - Next 24 Hours')
        # plt.xlabel('Hour')
        # plt.ylabel('Load (MW)')
        # plt.legend()
        # plt.grid(True)
        # plt.savefig(f"{state_name}_forecast.png")
        # plt.close()
        
        # Return predictions
        return {
            'state': state_name,
            'lstm_predictions': lstm_predictions,
            'fuzzy_inference': fuzzystrs
        }
    
    except Exception as e:
        print(f"Error in forecasting for {state_name}: {e}")
        return None


In [15]:

# 5. Main Function
def main():
    """
    Main function to run the load forecasting system.
    """
    try:
        print("Starting Load Forecasting System...")
        print(f"Current time: {datetime.datetime.now()}")
        
        # Load the data
        file_path = 'export_data.csv.txt' 
        data = load_data(file_path)
        
        if data is None:
            print("Failed to load data. Exiting.")
            return
        
        # Get list of states
        states = data['State'].unique()
        
        # Select a state for forecasting
        state_to_forecast = 'PUNJAB'  # Change this to forecast for a different state
        
        if state_to_forecast not in states:
            print(f"State {state_to_forecast} not found in data. Available states: {states}")
            state_to_forecast = states[0]
            print(f"Using {state_to_forecast} instead.")
        
        # Forecast load for the selected state
        forecast_results = forecast_load_for_state(data, state_to_forecast)
        
        if forecast_results:
            print("\nForecast Results:")
            print(f"State: {forecast_results['state']}")
            print("Hour | LSTM Prediction | Fuzzy Logic Refined")
            print("-" * 50)
            
            for hour, (lstm_pred, fuzzy_pred) in enumerate(zip(
                forecast_results['lstm_predictions'], 
                forecast_results['fuzzy_inference'], 
            )):
                print(f"{hour:2d}   | {lstm_pred:14.2f} | {fuzzy_pred}")
        
        print("\nLoad Forecasting System completed successfully!")
    
    except Exception as e:
        print(f"Error in main function: {e}")

if __name__ == "__main__":
    main()


Starting Load Forecasting System...
Current time: 2025-04-15 17:21:23.777672
Data loaded successfully with 29752 rows and 5 columns.

Data summary:
Time period: 2025-02-11 18:37:29.166139 to 2025-03-26 16:21:12.683044
States in the dataset: ['CHANDIGARH' 'HARYANA' 'HIMACHAL' 'J & K' 'PUNJAB' 'RAJASTHAN'
 'UTTARAKHAND' 'UTTAR-PRADESH']
Load value range: 83 to 21262

Building LSTM model for PUNJAB...
Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0550 - val_loss: 0.0035
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0039 - val_loss: 0.0036
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0027 - val_loss: 0.0016
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0020 - val_loss: 0.0012
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0016 - val_loss: 0.0010
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0014 - val_loss: 9.2

In [ ]:
forecast_results = forecast_load_for_state(data, state_to_forecast)